In [13]:
import random
import numpy as np
import math
import copy
import seaborn as sns

In [14]:
class ImportantValue:
    def __init__(self, position, value):
        self.position = position
        self.value = value

In [15]:
def TestFitness(array):
    return sum(array)

In [16]:
def BukinN6(array):
    under_sqrt = abs(array[1] - 0.01 * math.pow(array[0], 2))
    return 100*math.sqrt(under_sqrt) + 0.01 * abs(array[0] + 10)

In [17]:
def CrossInTray(array):
    exp_part =  math.exp(abs(100 - math.sqrt(pow(array[0],2) + pow(array[1], 2))/ math.pi))
    return -0.0001 * math.pow(abs(math.sin(array[0]) * math.sin(array[1]) * exp_part) + 1, 0.1)

In [18]:
def Eggholder(array):
    first_part = -(array[1] + 47) * math.sin(math.sqrt(abs(array[1] + array[0]/2 + 47)))
    second_part = array[0] * math.sin(math.sqrt(abs(array[0] - (array[1]+47))))
    return first_part - second_part

In [19]:
def DropWave(array):
    first_part = 1 + math.cos(12*math.sqrt(math.pow(array[0], 2) + math.pow(array[1],2)))
    second_part = 0.5 * (math.pow(array[0], 2) + math.pow(array[1], 2)) + 2
    return - (first_part / second_part)

In [20]:
def Greiwank(array):
    sum_part = sum(map(lambda x : math.pow(x, 2) / 4000, array))
    mul_part = 1
    for i in range(1, len(array)+1):
        #mul_part *= array[i]/math.sqrt(i)
        mul_part = mul_part * math.cos(array[i-1]/math.sqrt(i))
    return sum_part - mul_part + 1

In [21]:
def UpdateBestValues(positions_of_wolves, fitness_function, alpha, beta, delta):
    for wolf in positions_of_wolves:
        
            wolfs_fitness = fitness_function(wolf)
            
            if(wolfs_fitness < alpha.value):
                delta.value = beta.value
                delta.position = beta.position.copy()#copy.deepcopy(beta.position)
                
                beta.value = alpha.value
                beta.position = alpha.position.copy()#copy.deepcopy(alpha.position)
                
                alpha.value = wolfs_fitness
                alpha.position = wolf.copy()#copy.deepcopy(wolf)
                
            if(wolfs_fitness > alpha.value and wolfs_fitness < beta.value):
                delta.value = beta.value
                delta.position = beta.position.copy()#copy.deepcopy(beta.position)
                
                beta.value = wolfs_fitness
                beta.position = wolf.copy()#copy.deepcopy(wolf)
                
            if(wolfs_fitness > alpha.value and wolfs_fitness > beta.value and 
               wolfs_fitness < delta.value):
                delta.value = wolfs_fitness
                delta.position = wolf.copy()#copy.deepcopy(wolf)
                

In [22]:
def CalculateBestPosition(wolf, dim, important_position, a):
    r1 = random.random()
    r2 = random.random()

    C = 2 * r2
    A = (2 * a * r1) - a

    Dx = (C * important_position.position[dim]) - wolf[dim]
    X = important_position.position[dim] - (A * Dx)
    #print X
    return X

In [23]:
def GWO(pack_size, number_of_dimensions, number_of_iterations, fitness_function, lower_bound, 
        upper_bound):
    
    convergency = []
    
    if not isinstance(lower_bound, list):
        lower_bound = [lower_bound] * number_of_dimensions
    if not isinstance(upper_bound, list):
        upper_bound = [upper_bound] * number_of_dimensions
    
    alpha_position = np.zeros(number_of_dimensions)
    alpha_value = float('inf')
    alpha = ImportantValue(alpha_position, alpha_value)
    
    beta_position = np.zeros(number_of_dimensions)
    beta_value = float('inf')
    beta = ImportantValue(beta_position, beta_value)
    
    delta_position = np.zeros(number_of_dimensions)
    delta_value = float('inf')
    delta = ImportantValue(delta_position, delta_value)
    
    positions_of_wolves = np.zeros((pack_size, number_of_dimensions))
                                       
    for i in range(number_of_dimensions):
        positions_of_wolves[:, i] = np.random.uniform(lower_bound[i], upper_bound[i], 
                                                pack_size)
    
    UpdateBestValues(positions_of_wolves, fitness_function, alpha, beta, delta)
    
    for iter in range(0, number_of_iterations):
        a = 2 - (2/number_of_iterations) * iter
        
        for wolf in positions_of_wolves:
            for dim in range(number_of_dimensions):
                X1 = CalculateBestPosition(wolf, dim, alpha, a)
                X2 = CalculateBestPosition(wolf, dim, beta, a)
                X3 = CalculateBestPosition(wolf, dim, delta, a)
                
                wolf[dim] = (X1 + X2 + X3) / 3
                
                wolf[dim] = np.clip(wolf[dim], lower_bound[dim], upper_bound[dim])
                
        
        UpdateBestValues(positions_of_wolves, fitness_function, alpha, beta, delta)
        convergency.append(alpha.value)
 
    return alpha, convergency

In [24]:
best_alpha = ImportantValue([0, 0], float('inf'))

#for i in range(0, 20):
alpha, convergency = GWO(10, 10, 20000, Greiwank, -600, 600)

# if(alpha.value < best_alpha.value):
#     best_alpha = alpha
print(alpha.value)
print(alpha.position)
#sns.color_palette("bright")
#sns.lineplot(data=convergency)

0.0
[ 3.29173088e-09 -9.92270780e-09 -3.21536092e-09  1.51547713e-08
 -1.84549516e-08  1.23483223e-09  1.25674601e-09  1.39742048e-08
 -1.97327603e-08 -6.69663429e-09]
